In [18]:
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo 
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd

In [19]:
#Raw content URL for the dataset
url = "https://raw.githubusercontent.com/hastighsh/Ellehacks_2024/main/Monthly_weather_report.csv"

#Specify the delimiter
delimiter = ','

#Read the data into a DataFrame
weather = pd.read_csv(url, delimiter=delimiter)

#Let's create a backup copy of the dataset
weather_backup = weather.copy()

In [20]:
print(weather.shape)

(577927, 8)


In [21]:
### check for the number of these missing values 
missing_values_count = weather.isna().sum().sum()

print(f"Total number of missing values: {missing_values_count}")

missing_values_by_feature = weather.apply(lambda x: (x == '?').sum())
print(missing_values_by_feature)

Total number of missing values: 1553827
id                              0
Time                            0
Monthly Average Temperature     0
Monthly Maximum Temperature     0
Monthly Minimum Temperature     0
Monthly Total Precepitation     0
Monthly Average Windspeed       0
Monthly Average Air Pressure    0
dtype: int64


In [22]:
# Count missing values in each column
missing_values = weather.isnull().sum()

print("Missing values in each column:")
print(missing_values)


Missing values in each column:
id                                   0
Time                                 0
Monthly Average Temperature     293126
Monthly Maximum Temperature     159465
Monthly Minimum Temperature     159816
Monthly Total Precepitation     226342
Monthly Average Windspeed       319710
Monthly Average Air Pressure    395368
dtype: int64


In [23]:
display(weather)

,id,Time,Monthly Average Temperature,Monthly Maximum Temperature,Monthly Minimum Temperature,Monthly Total Precepitation,Monthly Average Windspeed,Monthly Average Air Pressure
0,70326,1942-02-01,1.0,NaN,NaN,42.6,NaN,NaN
1,70326,1942-03-01,-5.0,-0.4,-12.4,12.6,NaN,NaN
2,70326,1942-04-01,3.0,6.6,-2.2,35.6,NaN,NaN
3,70326,1942-05-01,9.3,14.2,2.5,28.8,NaN,NaN
4,70326,1942-06-01,10.8,15.6,4.9,65.8,15.2,1010.0
...,...,...,...,...,...,...,...,...
577922,PABL0,2011-12-01,NaN,NaN,NaN,NaN,NaN,NaN
577923,PABL0,2012-01-01,NaN,NaN,NaN,NaN,NaN,NaN
577924,PABL0,2012-02-01,NaN,NaN,NaN,NaN,NaN,NaN
577925,PABL0,2012-03-01,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# Convert 'Time' column to datetime
weather['Time'] = pd.to_datetime(weather['Time'])

# Filter data for the years 2000 to 2015
weather_filtered = weather[(weather['Time'].dt.year >= 2000) & (weather['Time'].dt.year <= 2015)]

# print("Filtered Data:")
# print(weather_filtered)

In [25]:
display(weather_filtered)

,id,Time,Monthly Average Temperature,Monthly Maximum Temperature,Monthly Minimum Temperature,Monthly Total Precepitation,Monthly Average Windspeed,Monthly Average Air Pressure
695,70326,2000-01-01,-15.3,-10.5,-20.1,29.0,14.5,1006.7
696,70326,2000-02-01,-0.7,3.0,-4.3,20.0,14.4,998.2
697,70326,2000-03-01,-0.7,2.7,-4.1,9.0,17.0,995.8
698,70326,2000-04-01,1.8,6.8,-3.1,17.0,12.7,1007.5
699,70326,2000-05-01,6.0,11.9,0.2,32.0,12.4,1013.5
...,...,...,...,...,...,...,...,...
577922,PABL0,2011-12-01,NaN,NaN,NaN,NaN,NaN,NaN
577923,PABL0,2012-01-01,NaN,NaN,NaN,NaN,NaN,NaN
577924,PABL0,2012-02-01,NaN,NaN,NaN,NaN,NaN,NaN
577925,PABL0,2012-03-01,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Count missing values in each column
missing_values = weather_filtered.isnull().sum()

print("Missing values in each column:")
print(missing_values)

Missing values in each column:
id                                   0
Time                                 0
Monthly Average Temperature      68171
Monthly Maximum Temperature      56605
Monthly Minimum Temperature      56663
Monthly Total Precepitation      84111
Monthly Average Windspeed        72114
Monthly Average Air Pressure    112326
dtype: int64


In [27]:
df = weather_filtered.copy()

In [28]:
# Convert 'Time' column to datetime
df['Time'] = pd.to_datetime(df['Time'])

# Extract year from 'Time' column
df['Year'] = df['Time'].dt.year

# Group by year and calculate mean
yearly_means = df.groupby('Year').mean(numeric_only=True) 

# Function to fill missing values with corresponding year's mean
def fill_missing_with_year_mean(row):
    year = row['Year']
    for col in df.columns[4:]:  # Exclude 'Unnamed' columns and 'id'
        if pd.isnull(row[col]):
            row[col] = yearly_means.loc[year, col]
    return row

# Apply the function row-wise
df_filled = df.apply(fill_missing_with_year_mean, axis=1)

# Drop the 'Year' column
df_filled.drop('Year', axis=1, inplace=True)

print("DataFrame with Missing Values Filled:")
print(df_filled)

DataFrame with Missing Values Filled:
           id       Time  Monthly Average Temperature  \
695     70326 2000-01-01                        -15.3   
696     70326 2000-02-01                         -0.7   
697     70326 2000-03-01                         -0.7   
698     70326 2000-04-01                          1.8   
699     70326 2000-05-01                          6.0   
...       ...        ...                          ...   
577922  PABL0 2011-12-01                          NaN   
577923  PABL0 2012-01-01                          NaN   
577924  PABL0 2012-02-01                          NaN   
577925  PABL0 2012-03-01                          NaN   
577926  PABL0 2012-04-01                        -10.3   

        Monthly Maximum Temperature  Monthly Minimum Temperature  \
695                           -10.5                   -20.100000   
696                             3.0                    -4.300000   
697                             2.7                    -4.100000   
698  

In [29]:
# Count missing values in each column
missing_values = df_filled.isnull().sum()

print("Missing values in each column:")
print(missing_values)

Missing values in each column:
id                                  0
Time                                0
Monthly Average Temperature     68171
Monthly Maximum Temperature     56605
Monthly Minimum Temperature         0
Monthly Total Precepitation         0
Monthly Average Windspeed           0
Monthly Average Air Pressure        0
dtype: int64


In [30]:
display(df_filled)

,id,Time,Monthly Average Temperature,Monthly Maximum Temperature,Monthly Minimum Temperature,Monthly Total Precepitation,Monthly Average Windspeed,Monthly Average Air Pressure
695,70326,2000-01-01,-15.3,-10.5,-20.100000,29.000000,14.500000,1006.700000
696,70326,2000-02-01,-0.7,3.0,-4.300000,20.000000,14.400000,998.200000
697,70326,2000-03-01,-0.7,2.7,-4.100000,9.000000,17.000000,995.800000
698,70326,2000-04-01,1.8,6.8,-3.100000,17.000000,12.700000,1007.500000
699,70326,2000-05-01,6.0,11.9,0.200000,32.000000,12.400000,1013.500000
...,...,...,...,...,...,...,...,...
577922,PABL0,2011-12-01,NaN,NaN,7.080456,76.548122,12.106698,1015.465438
577923,PABL0,2012-01-01,NaN,NaN,7.992203,69.516697,11.740267,1015.623922
577924,PABL0,2012-02-01,NaN,NaN,7.992203,69.516697,11.740267,1015.623922
577925,PABL0,2012-03-01,NaN,NaN,7.992203,69.516697,11.740267,1015.623922


In [31]:
print(df.columns)

Index(['id', 'Time', 'Monthly Average Temperature',
       'Monthly Maximum Temperature', 'Monthly Minimum Temperature',
       'Monthly Total Precepitation', 'Monthly Average Windspeed',
       'Monthly Average Air Pressure', 'Year'],
      dtype='object')


In [32]:
display(df_filled)

,id,Time,Monthly Average Temperature,Monthly Maximum Temperature,Monthly Minimum Temperature,Monthly Total Precepitation,Monthly Average Windspeed,Monthly Average Air Pressure
695,70326,2000-01-01,-15.3,-10.5,-20.100000,29.000000,14.500000,1006.700000
696,70326,2000-02-01,-0.7,3.0,-4.300000,20.000000,14.400000,998.200000
697,70326,2000-03-01,-0.7,2.7,-4.100000,9.000000,17.000000,995.800000
698,70326,2000-04-01,1.8,6.8,-3.100000,17.000000,12.700000,1007.500000
699,70326,2000-05-01,6.0,11.9,0.200000,32.000000,12.400000,1013.500000
...,...,...,...,...,...,...,...,...
577922,PABL0,2011-12-01,NaN,NaN,7.080456,76.548122,12.106698,1015.465438
577923,PABL0,2012-01-01,NaN,NaN,7.992203,69.516697,11.740267,1015.623922
577924,PABL0,2012-02-01,NaN,NaN,7.992203,69.516697,11.740267,1015.623922
577925,PABL0,2012-03-01,NaN,NaN,7.992203,69.516697,11.740267,1015.623922


In [34]:
id_count = df_filled['id'].value_counts()
print(id_count)

id
70326    192
KCNU0    192
KMOD0    192
KHOB0    192
72639    192
        ... 
KBPP0      2
KHZR0      2
PAII0      1
K5W80      1
K20U0      1
Name: count, Length: 1362, dtype: int64
